In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# Initialize the transformer pipeline and broadcast it
model = pipeline("text-classification", 
                model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
                top_k=None)
model_broadcast = spark.sparkContext.broadcast(model)

StatementMeta(, 032ac8b1-5dbd-489f-b02c-5d0916b9c454, 3, Finished, Available)

2024-06-05 16:09:47.513478: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 16:09:53.653415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [2]:
reviews = spark.sql("SELECT * FROM P2.ecom_reviews")
display(reviews)

StatementMeta(, 032ac8b1-5dbd-489f-b02c-5d0916b9c454, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3ebc284d-d356-43cc-9e3a-4c8821c42c99)

In [3]:
title = "Por lo que me costaron están bien, pero nada mas (Los conseguí por 12€)"
body = """
Me los compré para usarlos en el trabajo porque estaban de oferta y por las buenas opiniones que tienen, había leído que tenían un sonido relativamente plano pero debe ser que los que me han llegado están mal, porque a mi me parece que no suenan tan bien como dicen, aún costando lo que cuestan.Los graves están muy realzados y enmascaran las frecuencias medias, los agudos no están mal pero el sonido global es demasiado oscuro y poco detallado. Voy a darles algo mas de tiempo por si cambia el sonido tras unas horas de uso.La construcción y calidad de los materiales pues no esta mal para el precio, pero es todo bastante plasticoso y crujen cuando los tocas, las almohadillas son finas pero son suficientes para que no molesten cuando los tienes puestos. La diadema si que puede llegar a ser algo molesta si no la pones en la posición adecuada.Todo esto lo digo sabiendo que son unos auriculares baratos, aunque por tener tantas opiniones buenas me esperaba que fuesen mejores. Si los vas a usan para la consola o para ver vídeos cumplen bien, para escuchar música de momento no, a no ser que te guste la música con muchos graves.Ya veré si me los quedo, porque han sido un poco decepcionantes (Tengo unos baratuchos del Lidl que me costaron menos y se oyen mejor que estos)
""".strip()
text = f"{title} {body[:500]}"
result = model(text)[0]
# Remove neutral label
result = [r for r in result if r["label"] != "neutral"]
# Get the label with the maximum score
print(result)
sentiment = max(result, key=lambda x: x['score'])['label']
1 if sentiment == "positive" else 0

StatementMeta(, 032ac8b1-5dbd-489f-b02c-5d0916b9c454, 5, Finished, Available)

[{'label': 'negative', 'score': 0.707578718662262}, {'label': 'positive', 'score': 0.12962964177131653}]


0

In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

def get_sentiment(title, body):
    model = model_broadcast.value
    if body is None:
        body = ''
    text = f"{title}. {body[:500]}"
    result = model(text)[0]
    # Remove neutral label
    result = [r for r in result if r["label"] != "neutral"]
    # Get the label with the maximum score
    sentiment = max(result, key=lambda x: x['score'])['label']
    return sentiment == "positive"

# Register the UDF
sentiment_udf = udf(get_sentiment, BooleanType())

StatementMeta(, 032ac8b1-5dbd-489f-b02c-5d0916b9c454, 6, Finished, Available)

In [5]:
def count_nulls(df, column_name):
    # Count the number of null values in the specified column
    return df.filter(df[column_name].isNull()).count()

StatementMeta(, 032ac8b1-5dbd-489f-b02c-5d0916b9c454, 7, Finished, Available)

In [6]:
from pyspark.sql.functions import when

reviews_with_sentiment = reviews.withColumn(
    "sentiment",
    when(reviews.rating > 3, True)
    .when(reviews.rating < 3, False)
    .otherwise(sentiment_udf(reviews.title, reviews.body))
)
display(reviews_with_sentiment)

StatementMeta(, 032ac8b1-5dbd-489f-b02c-5d0916b9c454, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, a9389e7a-fb79-47b2-94a1-19ce63188426)

In [14]:
delta_table_name = "ecom_reviews"
# reviews_with_sentiment.write.format("delta") \
#     .option("mergeSchema", "true") \
#     .mode("overwrite") \
#     .saveAsTable(delta_table_name)
reviews_with_sentiment.write.format("delta").mode("overwrite").option("overwriteSchema", True).saveAsTable(delta_table_name)


print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, 032ac8b1-5dbd-489f-b02c-5d0916b9c454, 10, Finished, Available)

DataFrame has been written to Delta table: ecom_reviews
